# Results Generation

This notebook processes the results of the 2016 model from [How inter-state amity and animosity complement migration networks to drive refugee flows: A multi-layer network analysis, 1991–2016](https://journals.plos.org/plosone/article?id=10.1371/journal.pone.0245712) into predicted refugee shares for Ukraine.

In [9]:
import pandas as pd
import re

In [21]:
df = pd.read_csv('/Users/brandonrose/Downloads/S1_File/Data_for_correlation network.csv')\
    [['ccode1','ccode2','state.destination.name','state.origin.name']]

In [22]:
ccode_lookup = dict(zip(df.ccode1,df['state.origin.name']))

ccode_lookup_str = {}
for kk, vv in ccode_lookup.items():
    ccode_lookup_str[str(kk)] = vv

Down is ccode1, across ccode2

In [23]:
ref_preds = pd.read_csv('../../MR-QAP-refugees/ref_flow_preds_2016.csv')
img_preds = pd.read_csv('../../MR-QAP-refugees/immig_flow_preds_2017.csv')

In [24]:
def proc_matrix(df):
    df.rename(columns={'Unnamed: 0': 'ccode1'}, inplace=True)
    df['origin'] = df.ccode1.apply(lambda x: ccode_lookup[x])
    df.rename(columns=ccode_lookup_str, inplace=True)    
    df.drop('ccode1', axis=1, inplace=True)
    df = df.set_index('origin')    
    return df

In [25]:
ref_preds = proc_matrix(ref_preds)
img_preds = proc_matrix(img_preds)

In [26]:
ref_preds[['Hungary','Poland','Moldova','Romania','Slovakia','Belarus','Russia']].loc['Ukraine'].sort_values(ascending=False)\
    .round(0).astype(int)

Russia      26807
Belarus       801
Hungary       711
Romania       663
Slovakia      627
Moldova       495
Poland        408
Name: Ukraine, dtype: int64

In [27]:
img_preds[['Hungary','Poland','Moldova','Romania','Slovakia','Belarus','Russia']].loc['Ukraine'].sort_values(ascending=False)\
    .round(0).astype(int)

Russia      211955
Belarus     183756
Poland      182532
Moldova     170532
Hungary      34464
Romania      31016
Slovakia     30324
Name: Ukraine, dtype: int64

In [28]:
ref_res = pd.DataFrame(ref_preds[['Hungary','Poland','Moldova','Romania','Slovakia']].loc['Ukraine'].sort_values(ascending=False)\
    .round(0).astype(int))
ref_res['share'] = ref_res['Ukraine'] / ref_res['Ukraine'].sum()
display(ref_res)

,Ukraine,share
Hungary,711,0.244835
Romania,663,0.228306
Slovakia,627,0.215909
Moldova,495,0.170455
Poland,408,0.140496


In [29]:
img_res = pd.DataFrame(img_preds[['Hungary','Poland','Moldova','Romania','Slovakia', 'Russia','Belarus']].loc['Ukraine'].sort_values(ascending=False)\
    .round(0).astype(int))
img_res['share'] = img_res['Ukraine'] / img_res['Ukraine'].sum()
display(img_res)

,Ukraine,share
Russia,211955,0.250959
Belarus,183756,0.217571
Poland,182532,0.216122
Moldova,170532,0.201914
Hungary,34464,0.040806
Romania,31016,0.036724
Slovakia,30324,0.035904


In [30]:
print("Predictions from refugee model")
display(ref_res)
print()
print()
print("Predictions from immigration model")
display(img_res)

Predictions from refugee model


,Ukraine,share
Hungary,711,0.244835
Romania,663,0.228306
Slovakia,627,0.215909
Moldova,495,0.170455
Poland,408,0.140496




Predictions from immigration model


,Ukraine,share
Russia,211955,0.250959
Belarus,183756,0.217571
Poland,182532,0.216122
Moldova,170532,0.201914
Hungary,34464,0.040806
Romania,31016,0.036724
Slovakia,30324,0.035904


In [31]:
ukr_results = img_res.join(ref_res, lsuffix='_immig', rsuffix='_ref')
ukr_results['predicted_shares'] = (ukr_results.share_immig + ukr_results.share_ref) / 2
ukr_results = ukr_results.reset_index().rename(columns={'index':'country'})
ukr_results = ukr_results[['country','predicted_shares']]
display(ukr_results)

,country,predicted_shares
0,Russia,NaN
1,Belarus,NaN
2,Poland,0.178309
3,Moldova,0.186184
4,Hungary,0.142820
5,Romania,0.132515
6,Slovakia,0.125907


In [32]:
preds.to_csv('../../MR-QAP-refugees/results.csv')

NameError: name 'preds' is not defined

In [ ]:
pd.DataFrame(preds.loc['Ukraine'])

In [ ]:
df_['year'] = df_['variable'].apply(lambda x: int(x[-4:]))

In [ ]:
ukr_migrants = df_[(df_['state.origin.name']=='Ukraine')&(df_['year']==2010)].sort_values(by='value',ascending=False)[:20][['state.destination.name','value']]

In [ ]:
pd.DataFrame(ukr_migrants.groupby(['state.destination.name']).value.sum()).round().astype(int).sort_values(by='value', ascending=False)